In [1]:
import findspark

In [2]:
findspark.init('/home/admin123/spark-2.4.0-bin-hadoop2.7/')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark  = SparkSession.builder.appName('lr-example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('Ecommerce-Customers.csv', inferSchema=True, header=True )

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [11]:
 for item in data.head(1)[0]:
        print (item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [13]:
from pyspark.ml.linalg import Vectors

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [17]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Length of Membership','Yearly Amount Spent'], outputCol='features')

In [18]:
output = assembler.transform(data)

In [19]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 4.0826, 587.9511]))]

In [21]:
final_data = output.select('features','Yearly Amount Spent')

In [22]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [23]:
training_data, test_data = final_data.randomSplit([0.7, 0.3])

In [24]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                351|
|   mean|   499.947374768234|
| stddev|  79.99581018096454|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [25]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                149|
|   mean|  497.8220844674169|
| stddev|  77.93312365571744|
|    min| 298.76200786180766|
|    max|  712.3963268096637|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
lr_model = lr.fit(training_data)

In [28]:
test_results = lr_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.602984411874786...|
|1.762145984685048...|
|3.183231456205249...|
|4.547473508864641...|
|2.569322532508522...|
|7.332801033044234...|
|2.393107934040017...|
|7.958078640513122...|
|2.273736754432320...|
|-1.70530256582424...|
|2.160049916710704...|
|1.938360583153553...|
|9.890754881780595...|
|-1.81898940354585...|
|-1.51771928358357...|
|-5.79802872380241...|
|1.023181539494544...|
|-1.30739863379858...|
|-5.05906427861191...|
|1.48929757415317E-11|
+--------------------+
only showing top 20 rows



In [30]:
test_results.rootMeanSquaredError

1.0119219415402025e-11

In [31]:
test_results.r2

1.0

In [32]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [35]:
unlabeled_data = test_data.select('features')

In [36]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[30.8794843441274...|
|[31.2606468698795...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.3123495994443...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.5147378578019...|
|[31.5761319713222...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.7207699002873...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9262720263601...|
+--------------------+
only showing top 20 rows



In [37]:
prediction = lr_model.transform(unlabeled_data)

In [41]:
#pridcitions
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|461.78074219621385|
|[30.8794843441274...| 490.2065999848529|
|[31.2606468698795...| 421.3266312569482|
|[31.2681042107507...|423.47053317382347|
|[31.2834474760581...| 591.7810894256418|
|[31.3091926408918...| 432.7207178399263|
|[31.3123495994443...| 463.5914180279167|
|[31.3584771924370...|495.17595044946745|
|[31.4252268808548...| 530.7667186547617|
|[31.5147378578019...| 489.8124879964616|
|[31.5761319713222...| 541.2265839893262|
|[31.6098395733896...| 444.5455496510888|
|[31.6548096756927...| 475.2634237275386|
|[31.7207699002873...| 538.7749334780248|
|[31.8164283341993...|501.12249150367154|
|[31.8279790554652...| 440.0027475469473|
|[31.8530748017465...|459.28512346235095|
|[31.8745516945853...| 392.2852442462688|
|[31.8854062999117...|390.10327297248057|
|[31.9262720263601...| 392.2049334443115|
+--------------------+------------